In [23]:
import requests
from smolagents import CodeAgent, tool, LiteLLMModel



In [26]:
import requests
from smolagents import CodeAgent, tool, LiteLLMModel

# 1. Define the Transit Tool
@tool
def get_paris_itinerary(start: str, destination: str) -> str:
    """
    Calculates the best public transport route between two points in Paris.
    Args:
        start: The starting address or station name.
        destination: The destination address or station name.
    """
    # Replace with your actual Navitia API key
    api_key = "YOUR_NAVITIA_API_TOKEN"
    url = f"https://api.navitia.io/v1/coverage/fr-idf/journeys?from={start}&to={destination}"
    
    response = requests.get(url, auth=(api_key, ""))
    if response.status_code != 200:
        return "Error fetching data from Navitia."
    
    data = response.json()
    # Simplified extraction logic:
    journey = data['journeys'][0]
    duration = journey['duration'] // 60
    sections = []
    for s in journey['sections']:
        if 'display_informations' in s:
            line = s['display_informations']['label']
            sections.append(f"Take {line}")
            
    return f"Trip takes {duration} mins. Route: {' -> '.join(sections)}"

@tool
def check_disruptions(line_name: str) -> str:
    """
    Checks for real-time traffic disruptions on a specific Paris Metro/RER line.
    Args:
        line_name: The line to check (e.g., 'Metro 1', 'RER A').
    """
    # Similar API call to Navitia's /line_reports endpoint
    return f"No major disruptions currently reported for {line_name}."


In [ ]:

# 2. Setup the LLM (Using a free model via Groq or Hugging Face)
model = LiteLLMModel(model_id="groq/llama-3.1-70b-versatile")

# 3. Initialize the Agent
agent = CodeAgent(tools=[get_paris_itinerary, check_disruptions], model=model)

# 4. Run the Assistant
agent.run("I am at Châtelet and want to go to Eiffel Tower. Give me the itinerary and check if there are issues on the lines.")

In [21]:
import requests

def get_all_metro_stations(api_key: str):
    """
    Retrieves all Paris Metro stations (stop_areas) filtered by commercial mode.
    """
    # The URI for Metro is 'commercial_mode:idfm:Metro'
    url='https://prim.iledefrance-mobilites.fr/marketplace/v2/navitia/commercial_modes/commercial_mode:Metro/stop_areas'
    
    headers = {"apikey": api_key}
    # count is used to handle pagination; set high to get as many as possible per request
    params = {"count": 1000} 

    response = requests.get(url, headers=headers,params=params)
    if response.status_code != 200:
        return f"Error: {response.status_code}"

    data = response.json()
    metro_stations = []

    for station in data.get("stop_areas", []):
        entry = {
            "id": station["id"],
            "name": station["name"],
            "coordinates": {
                "lat": station["coord"]["lat"],
                "lon": station["coord"]["lon"]
            }
        }
        metro_stations.append(entry)

    return metro_stations

# Example Usage:
stations_list = get_all_metro_stations("RO29yT8rnY192fRyHgE2DV57jiQWXOCI")
print(f"Found {len(stations_list)} Metro stations.")

Found 321 Metro stations.


In [22]:
stations_list

[{'id': 'stop_area:IDFM:71432',
  'name': 'Abbesses',
  'coordinates': {'lat': '48.884608', 'lon': '2.338228'}},
 {'id': 'stop_area:IDFM:63284',
  'name': 'Aéroport d’Orly (Terminaux 1-2-3)',
  'coordinates': {'lat': '48.728628', 'lon': '2.360904'}},
 {'id': 'stop_area:IDFM:72491',
  'name': 'Aimé Césaire',
  'coordinates': {'lat': '48.90826', 'lon': '2.377751'}},
 {'id': 'stop_area:IDFM:71030',
  'name': 'Alésia',
  'coordinates': {'lat': '48.828182', 'lon': '2.326848'}},
 {'id': 'stop_area:IDFM:71728',
  'name': 'Alexandre Dumas',
  'coordinates': {'lat': '48.856314', 'lon': '2.394229'}},
 {'id': 'stop_area:IDFM:71290',
  'name': 'Alma - Marceau',
  'coordinates': {'lat': '48.864569', 'lon': '2.301562'}},
 {'id': 'stop_area:IDFM:71519',
  'name': 'Anatole France',
  'coordinates': {'lat': '48.89181', 'lon': '2.284864'}},
 {'id': 'stop_area:IDFM:71419',
  'name': 'Anvers',
  'coordinates': {'lat': '48.882868', 'lon': '2.344134'}},
 {'id': 'stop_area:IDFM:71348',
  'name': 'Argentine',

In [11]:
import requests

def get_station_details(station_name: str, api_key: str):
    """
    Calls the /places endpoint and extracts only ID, Name, and Coordinates.
    """
    url = "https://prim.iledefrance-mobilites.fr/marketplace/v2/navitia/places"
    headers = {"apikey": api_key}
    params = {
        "q": station_name,
        "type[]": "stop_area", # Only return transport hubs, not cafes or addresses
        "count": 5             # Return top 5 matches to allow the agent to choose
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return "Error connecting to API"

    data = response.json()
    cleaned_stations = []

    for place in data.get("places", []):
        # We only want the Stop Areas (Stations)
        if place["embedded_type"] == "stop_area":
            station_info = {
                "id": place["id"],
                "name": place["name"],
                "coordinates": {
                    "lat": place["stop_area"]["coord"]["lat"],
                    "lon": place["stop_area"]["coord"]["lon"]
                }
            }
            cleaned_stations.append(station_info)

    return cleaned_stations

# Example Call:
stations = get_station_details("Châtelet", "RO29yT8rnY192fRyHgE2DV57jiQWXOCI")
print(stations)

[{'id': 'stop_area:IDFM:71264', 'name': 'Châtelet (Paris)', 'coordinates': {'lat': '48.85826227294274', 'lon': '2.3482877360417778'}}, {'id': 'stop_area:IDFM:474151', 'name': 'Châtelet les Halles (Paris)', 'coordinates': {'lat': '48.86174503191322', 'lon': '2.34697651043533'}}, {'id': 'stop_area:IDFM:61428', 'name': 'Châtelet (Montereau-Fault-Yonne)', 'coordinates': {'lat': '48.38370714957764', 'lon': '2.944131196335433'}}, {'id': 'stop_area:IDFM:61222', 'name': 'Petit Châtelet (Darvault)', 'coordinates': {'lat': '48.27026199901867', 'lon': '2.7268901202756997'}}, {'id': 'stop_area:IDFM:64543', 'name': 'Châtelet / Sécurité Sociale (Chatou)', 'coordinates': {'lat': '48.893819708363615', 'lon': '2.1582030763069775'}}]


In [9]:
import requests
import json

def build_station_database():
    api_key = "RO29yT8rnY192fRyHgE2DV57jiQWXOCI"
    # coverage/idfm limits the search to the Paris region
    url = "https://prim.iledefrance-mobilites.fr/marketplace/v2/navitia/stop_areas"
    
    headers = {"apikey": api_key}
    params = {"count": 10000} # Get a large batch
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    simple_db = []
    
    for station in data.get('stop_areas', []):
        entry = {
            "id": station.get("id"),
            "name": station.get("name"),
            "lat": station.get("coord", {}).get("lat"),
            "lon": station.get("coord", {}).get("lon")
        }
        simple_db.append(entry)
        
    with open("paris_stations.json", "w", encoding="utf-8") as f:
        json.dump(simple_db, f, indent=4, ensure_ascii=False)
        
    print(f"Stored {len(simple_db)} stations.")

build_station_database()

Stored 1000 stations.


In [2]:
def get_paris_itinerary(start: str, destination: str) -> str:
    """
    Calculates the best public transport route between two points in Paris.
    Args:
        start: The starting address or station name.
        destination: The destination address or station name.
    """
    # Replace with your actual Navitia API key
    api_key = "RO29yT8rnY192fRyHgE2DV57jiQWXOCI"
    headers = {
    "apikey": api_key
}
    #url = f"https://api.navitia.io/v1/coverage/fr-idf/journeys?from={start}&to={destination}"
    url=f"https://api.navitia.io/v1/coverage/us-ca/journeys?from=-122.4752;37.80826&to=-122.402770;37.794682"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Error fetching data from Navitia."
    
    data = response.json()
    # Simplified extraction logic:
    journey = data['journeys'][0]
    duration = journey['duration'] // 60
    sections = []
    for s in journey['sections']:
        if 'display_informations' in s:
            line = s['display_informations']['label']
            sections.append(f"Take {line}")
            
    return f"Trip takes {duration} mins. Route: {' -> '.join(sections)}"


In [4]:
get_paris_itinerary("Gare de Lyon", "Châtelet")

'Error fetching data from Navitia.'

In [ ]:

# 1. Define the Transit Tool
@tool
def get_paris_itinerary(start: str, destination: str) -> str:
    """
    Calculates the best public transport route between two points in Paris.
    Args:
        start: The starting address or station name.
        destination: The destination address or station name.
    """
    # Replace with your actual Navitia API key
    api_key = "YOUR_NAVITIA_API_TOKEN"
    url = f"https://api.navitia.io/v1/coverage/fr-idf/journeys?from={start}&to={destination}"
    
    response = requests.get(url, auth=(api_key, ""))
    if response.status_code != 200:
        return "Error fetching data from Navitia."
    
    data = response.json()
    # Simplified extraction logic:
    journey = data['journeys'][0]
    duration = journey['duration'] // 60
    sections = []
    for s in journey['sections']:
        if 'display_informations' in s:
            line = s['display_informations']['label']
            sections.append(f"Take {line}")
            
    return f"Trip takes {duration} mins. Route: {' -> '.join(sections)}"

@tool
def check_disruptions(line_name: str) -> str:
    """
    Checks for real-time traffic disruptions on a specific Paris Metro/RER line.
    Args:
        line_name: The line to check (e.g., 'Metro 1', 'RER A').
    """
    # Similar API call to Navitia's /line_reports endpoint
    return f"No major disruptions currently reported for {line_name}."

# 2. Setup the LLM (Using a free model via Groq or Hugging Face)
model = LiteLLMModel(model_id="groq/llama-3.1-70b-versatile")

# 3. Initialize the Agent
agent = CodeAgent(tools=[get_paris_itinerary, check_disruptions], model=model)

# 4. Run the Assistant
agent.run("I am at Châtelet and want to go to Eiffel Tower. Give me the itinerary and check if there are issues on the lines.")